In [2]:
import psutil
psutil.cpu_percent()

0.0

In [ ]:
# 2.87 s ± 13 ms per loop

import psutil
import time
import os
from IPython.display import clear_output

current_pid = os.getpid()
print(f'setting high prio for set_caiman_nice.ipynb. pid = {current_pid}')
psutil.Process(os.getpid()).nice(psutil.ABOVE_NORMAL_PRIORITY_CLASS)

while 1>0: # forever set caiman nice
# while psutil.cpu_percent() > 50: # when cpu too high, set caiman nice
    cpu_util = psutil.cpu_percent()
    print(f'cpu util = {cpu_util}, start setting caiman nice')

    process_name = "python"
    pids = []
    for proc in psutil.process_iter():
        if process_name in proc.name():
            pids.append(proc.pid)
    pids.remove(os.getpid()) # get pid for all python processes except for self
    
    print(f'setting low prio for all caiman process.')
    for i in pids:
        print(f'pid = {i}')
        psutil.Process(i).nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)

    time.sleep(600) # check for new python process every n sec
    clear_output(wait=True) # clear cell output after a pause, otherwise too long

cpu util = 0.7, start setting caiman nice
setting low prio for all caiman process.
pid = 632
pid = 1760
pid = 2264
pid = 2920
pid = 3172
pid = 4640
pid = 4704
pid = 4896
pid = 6128
pid = 8888
pid = 9372
pid = 9396
pid = 9816
pid = 10212
pid = 10812
pid = 11228
pid = 11336
pid = 11380
pid = 11540
pid = 11876
pid = 12196
pid = 12384
pid = 12908
pid = 12928
pid = 13236
pid = 13548
pid = 13848
pid = 14512
pid = 14736
pid = 15000
pid = 15188
pid = 15396
pid = 15460
pid = 15620
pid = 15748
pid = 15828
pid = 16024
pid = 16064
pid = 16136
pid = 16660
pid = 16796
pid = 16816
pid = 16936
pid = 17304
pid = 17376
pid = 18060
pid = 18116
pid = 18136
pid = 18216
pid = 18516
pid = 18704
pid = 18920
pid = 19064
pid = 19488
pid = 19504
pid = 19588
pid = 19996
pid = 20740
pid = 20988
pid = 21288
pid = 21372
pid = 21584
pid = 22216
pid = 22568
pid = 22752
pid = 22816
pid = 23020
pid = 23112
pid = 23600
pid = 24140
pid = 24500
pid = 24512
pid = 24640
pid = 25076
pid = 25272
pid = 25520
pid = 25596
pid = 2